In [1]:
%matplotlib inline

# Exercise 4

Multiple observations may be used in models.  What proportion of days where I am 5 minutes late (`total_time=65`) *and* rode my bike (`takes_bike=1`) does the train take more than 45 minutes (`train_time`)?

In [2]:
import pymc3 as pm
from sampled import sampled
import theano.tensor as tt

In [3]:
@sampled
def commute():
    train_time = pm.SkewNormal('train_time', mu=40., sd=10., alpha=15.)
    takes_bike = pm.Binomial('takes_bike', n=1, p=0.1)
    bike_time = pm.Normal('bike_time', mu=20., sd=3.)
    walk_time = pm.Normal('walk_time', mu=5., sd=1.)
    t_time = pm.SkewNormal('t_time', mu=15., sd=5., alpha=4.)
    total_time = pm.Normal('total_time', 
                           mu=train_time + tt.switch(takes_bike, bike_time, walk_time+t_time), 
                           sd=1)

In [4]:
with commute(total_time=65, takes_bike=1):
    trace = pm.sample()

print((trace['train_time'] > 45).mean())

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
100%|██████████| 1000/1000 [00:02<00:00, 436.58it/s]


0.49
